# Gold

## Customer

* SDC2 Dimension

In [0]:
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable

In [0]:
# reading silver and gold tables 
df_silver = DeltaTable.forName(spark, "silver.Customers").toDF()
df_gold = DeltaTable.forName(spark, "gold.Customers")

# Mark 1 for delete and 3 for update (before image) in silver.table
df_gold.alias("target").merge(
    df_silver.alias("source"),
    "target.CUSTOMER_ID == source.CUSTOMER_ID AND target.active == 1"
).whenMatchedUpdate(
    condition="target.hash != source.hash AND target.active == 1",
    set={
        "active": "0",
        "end_timestamp": current_timestamp()
    }
).whenNotMatchedBySourceUpdate(
    condition="target.active == 1",
    set={
        "active": "0",
        "end_timestamp": current_timestamp()
    }
).execute()

# Insert Updated records to gold table
df_gold.alias("target").merge(
    df_silver.alias("source"),
    "target.CUSTOMER_ID == source.CUSTOMER_ID AND target.active == 1"
).whenNotMatchedInsert(
    values = {
        "CUSTOMER_ID": "source.CUSTOMER_ID",
        "hash": "source.hash",
        "load_timestamp": "source.load_timestamp",
        "end_timestamp": "source.end_timestamp",
        "active": "1",
        "Social_Security_Number": "source.Social_Security_Number",
        "Name": "source.Name",
        "Email": "source.Email",
        "Phone": "source.Phone",
        "Address": "source.Address",
        "Zip_Code": "source.Zip_Code"
        }
).execute()

In [0]:
%sql
